# 05 · Evaluation Generation & Log-probs

Load Unsloth checkpoints, run evaluation prompts, and capture per-token statistics for MIA features.

In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.

In [ ]:
# Persistent Drive + run mode setup
import os
import sys
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    DRIVE_MOUNT = Path('/content/drive')
    if not DRIVE_MOUNT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if Path('/content/drive').exists():
    DRIVE_ROOT = Path('/content/drive/MyDrive').resolve()
else:
    DRIVE_ROOT = Path.home().resolve()

PROJECT_ROOT = DRIVE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Run 00_colab_setup.ipynb first to clone the repo on Drive.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from src.utils.runtime import current_run_mode

RUN_MODE = current_run_mode()
print('PROJECT_ROOT:', PROJECT_ROOT)
print('Active run mode:', RUN_MODE.name, '-', RUN_MODE.description)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

BHC_DATA_DIR = DRIVE_ROOT / 'mimic-iv-bhc'
BHC_DATA_DIR.mkdir(parents=True, exist_ok=True)
BHC_CSV_PATH = BHC_DATA_DIR / 'mimic-iv-bhc.csv'
print('BHC CSV path:', BHC_CSV_PATH)


Mounted at /content/drive
PROJECT_ROOT: /content/drive/MyDrive/secure-llm-mia
Active run mode: subset - Quick debugging subset (<=2k rows) for lightweight Colab smoke tests.
BHC CSV path: /content/drive/MyDrive/mimic-iv-bhc/mimic-iv-bhc.csv


In [ ]:
import numpy as np
import torch
from datasets import Dataset
from unsloth import FastLanguageModel

from src.modeling.logprobs import token_level_stats

MAX_SEQ_LENGTH = 4096
EVAL_PER_CLASS = 5  # number of member/non-member examples to score per slice
FEATURES_DIR = PROJECT_ROOT / 'reports' / 'features'
FEATURES_DIR.mkdir(parents=True, exist_ok=True)


def _discover_slice_ids(checkpoint_root: Path) -> list[int]:
    slice_ids: list[int] = []
    for path in checkpoint_root.glob('slice_*'):
        parts = path.name.split('_')
        if len(parts) != 2:
            continue
        try:
            slice_ids.append(int(parts[1]))
        except ValueError:
            continue
    return sorted(set(slice_ids))


def _discover_tracks(checkpoint_root: Path, slice_ids: list[int]) -> list[str]:
    tracks: set[str] = set()
    for slice_id in slice_ids:
        slice_dir = checkpoint_root / f'slice_{slice_id}'
        if not slice_dir.exists():
            continue
        for child in slice_dir.iterdir():
            if child.is_dir():
                tracks.add(child.name)
    return sorted(tracks)


SLICES = _discover_slice_ids(CHECKPOINT_ROOT)
if not SLICES:
    raise ValueError('No slice checkpoints found. Run notebook 04 first.')

TRACKS = _discover_tracks(CHECKPOINT_ROOT, SLICES)
if not TRACKS:
    raise ValueError('No replay tracks found under checkpoints.')

print('Will evaluate slices:', SLICES)
print('Replay tracks:', TRACKS)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

unsloth/meta-llama-3.1-8b-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import pandas as pd

CANONICAL_PATH = ARTIFACTS_DIR / f'canonical_bhc_{RUN_MODE.name}.parquet'
if not CANONICAL_PATH.exists():
    raise FileNotFoundError('Canonical dataset missing. Run notebook 01 to create it.')

canonical_df = pd.read_parquet(CANONICAL_PATH)
canonical_df = canonical_df.drop_duplicates('subject_id').set_index('subject_id')
print('Canonical rows available:', len(canonical_df))


Evaluation examples: 10 (members=5, non-members=5)


In [5]:
TOP_K_VALUES = (1, 5, 10, 20)


def _read_ids(path: Path) -> list[str]:
    if not path.exists():
        return []
    return [line.strip() for line in path.read_text().splitlines() if line.strip()]


def _collect_eval_examples(slice_id: int, per_class: int) -> list[dict]:
    ids_dir = ARTIFACTS_DIR / f'slice_{slice_id}' / 'ids'
    member_ids = _read_ids(ids_dir / 'members.txt')
    non_member_ids = _read_ids(ids_dir / 'non_members.txt')
    if not member_ids or not non_member_ids:
        return []
    limit = min(per_class, len(member_ids), len(non_member_ids))
    if limit == 0:
        return []
    examples: list[dict] = []
    for sid in member_ids[:limit]:
        if sid in canonical_df.index:
            examples.append({'example_id': sid, 'label': 1, 'text': canonical_df.at[sid, 'text']})
    for sid in non_member_ids[:limit]:
        if sid in canonical_df.index:
            examples.append({'example_id': sid, 'label': 0, 'text': canonical_df.at[sid, 'text']})
    return examples


for track in TRACKS:
    print(f'=== Track: {track} ===')
    for slice_id in SLICES:
        checkpoint_dir = CHECKPOINT_ROOT / f'slice_{slice_id}' / track / RUN_MODE.name
        if not checkpoint_dir.exists():
            print(f'Skipping slice {slice_id} ({track}) - checkpoint missing: {checkpoint_dir}')
            continue

        eval_examples = _collect_eval_examples(slice_id, EVAL_PER_CLASS)
        if not eval_examples:
            print(f'Skipping slice {slice_id} ({track}) - ID lists missing or empty.')
            continue

        print(f'Slice {slice_id}: evaluating {len(eval_examples)} examples ({track}).')
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=str(checkpoint_dir),
            max_seq_length=MAX_SEQ_LENGTH,
            load_in_4bit=True,
        )
        FastLanguageModel.for_inference(model)
        tokenizer.model_max_length = MAX_SEQ_LENGTH

        example_ids: list[str] = []
        labels: list[int] = []
        stats_list: list[dict] = []

        for example in eval_examples:
            inputs = tokenizer(
                example['text'],
                return_tensors='pt',
                padding=True,
                truncation=True,
                max_length=MAX_SEQ_LENGTH,
            ).to(model.device)
            with torch.inference_mode():
                outputs = model(**inputs)
            logits = outputs.logits.detach().cpu().to(torch.float32).numpy()
            target_ids = inputs['input_ids'].cpu().numpy()
            stats_list.append(token_level_stats(logits, target_ids))
            example_ids.append(example['example_id'])
            labels.append(example['label'])

        if not stats_list:
            print(f'Skipping slice {slice_id} ({track}) - no stats computed.')
            del model, tokenizer
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            continue

        feature_data = {
            'example_id': example_ids,
            'label': labels,
            'token_nll': [s['nll'].flatten().tolist() for s in stats_list],
            'token_entropy': [s['entropy'].flatten().tolist() for s in stats_list],
            'token_max_prob': [s['max_prob'].flatten().tolist() for s in stats_list],
        }
        for k in TOP_K_VALUES:
            key = f'win@{k}'
            feature_data[key] = [s[key].flatten().astype(int).tolist() for s in stats_list]

        features_path = FEATURES_DIR / f'features_slice_{slice_id}_{track}_{RUN_MODE.name}.parquet'
        Dataset.from_dict(feature_data).to_parquet(str(features_path))
        print(f'Saved feature dataset to {features_path}')

        del model, tokenizer
        if torch.cuda.is_available():
            torch.cuda.empty_cache()


=== Track: noreplay ===
Slice 1: evaluating 10 examples (noreplay).
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

unsloth/llama-3.2-3b-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Unsloth 2025.11.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


...

Saved feature dataset to /content/drive/MyDrive/secure-llm-mia/reports/features/features_slice_2_replay10_subset.parquet
Slice 3: evaluating 10 examples (replay10).
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
unsloth/llama-3.2-3b-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Saved feature dataset to /content/drive/MyDrive/secure-llm-mia/reports/features/features_slice_3_replay10_subset.parquet
Slice 4: evaluating 10 examples (replay10).
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
unsloth/llama-3.2-3b-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Saved feature dataset to /content/drive/MyDrive/secure-llm-mia/reports/features/features_slice_4_replay10_subset.parquet
